In [35]:
import loralib as lora
from transformers import AutoModel, AutoTokenizer
from transformers import GPT2LMHeadModel

import torch
import torch.nn as nn

In [36]:
model_name_or_path = "pretrained_model/gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [37]:
txt = "你好，今天是星期几？"
input_embedding = tokenizer(txt, return_tensors="pt")
input_embedding

{'input_ids': tensor([[ 101,  872, 1962, 8024,  791, 1921, 3221, 3215, 3309, 1126, 8043,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [41]:
def print_trainable_params(model):
    """打印模型可训练参数量占比

    Args:
        model (_type_): _description_

    """
    total_params = sum([p.numel() for p in model.parameters()])
    trainable_params = sum([p.numel() for p in model.parameters() if p.requires_grad])
    trainable_ratio = trainable_params / total_params
    print(f'total parameters: {total_params} || trainable parameters: {trainable_params} || \
    trainable ratio: {100*trainable_ratio:.2f}%')

In [42]:
print_trainable_params(model)

total parameters: 102068736 || trainable parameters: 102068736 ||     trainable ratio: 100.00%


In [46]:
for layer in model.transformer.h:
    layer.attn.c_attn = lora.Conv1d(768, 768, 1, r=4)
    layer.attn.c_proj = lora.Conv1d(768, 768, 1, r=4)

lora.mark_only_lora_as_trainable(model)
print_trainable_params(model)

total parameters: 88041984 || trainable parameters: 147456 ||     trainable ratio: 0.17%


In [ ]:
cur_layer = lora.conv1d()